In [1]:
from thesis_v2.analysis.utils import LayerSourceAnalysis, get_source_analysis_for_one_model_spec
from collections import defaultdict

In [2]:
# 7 iteration network, `sources` gives the contribution at each time step,

# this network assumes that the bottom up input never changes,
# and the bottom up input is processed using conv layer `B`
# and the lateral input is processed using conv layer `R`
# at the end of time t (k=1,2,3,4,5,6,7), we apply a BN layer with (average) scale `s{t}`.
# bias, activation layers are not explictly represented here and will be ignored during analysis


sources = [LayerSourceAnalysis().add_source(conv=('B',), scale=('s1',))]

for t in range(2, 7+1):
    src_this = LayerSourceAnalysis().add_source(
        conv=('B',), scale=()
    )
    # input from recurrent
    src_prev_with_recurrent = sources[-1].apply_conv(('R',))
    
    src_this = src_this.add(src_prev_with_recurrent)
    src_this = src_this.apply_scale((f's{t}',))
    
    sources.append(src_this)

In [3]:
[s.source_list for s in sources]
# this looks right, the same as my handwritten analysis.

[[{'conv': ('B',), 'scale': ('s1',)}],
 [{'conv': ('B',), 'scale': ('s2',)},
  {'conv': ('B', 'R'), 'scale': ('s1', 's2')}],
 [{'conv': ('B',), 'scale': ('s3',)},
  {'conv': ('B', 'R'), 'scale': ('s2', 's3')},
  {'conv': ('B', 'R', 'R'), 'scale': ('s1', 's2', 's3')}],
 [{'conv': ('B',), 'scale': ('s4',)},
  {'conv': ('B', 'R'), 'scale': ('s3', 's4')},
  {'conv': ('B', 'R', 'R'), 'scale': ('s2', 's3', 's4')},
  {'conv': ('B', 'R', 'R', 'R'), 'scale': ('s1', 's2', 's3', 's4')}],
 [{'conv': ('B',), 'scale': ('s5',)},
  {'conv': ('B', 'R'), 'scale': ('s4', 's5')},
  {'conv': ('B', 'R', 'R'), 'scale': ('s3', 's4', 's5')},
  {'conv': ('B', 'R', 'R', 'R'), 'scale': ('s2', 's3', 's4', 's5')},
  {'conv': ('B', 'R', 'R', 'R', 'R'),
   'scale': ('s1', 's2', 's3', 's4', 's5')}],
 [{'conv': ('B',), 'scale': ('s6',)},
  {'conv': ('B', 'R'), 'scale': ('s5', 's6')},
  {'conv': ('B', 'R', 'R'), 'scale': ('s4', 's5', 's6')},
  {'conv': ('B', 'R', 'R', 'R'), 'scale': ('s3', 's4', 's5', 's6')},
  {'conv':

In [4]:
scale_map = {
    's1': 1.0,
    's2': 2.0,
    's3': 3.0,
    's4': 4.0,
    's5': 5.0,
    's6': 6.0,
    's7': 7.0,
}

In [5]:
[s.evaluate(scale_map) for s in sources]

[{('B',): 1.0},
 {('B',): 2.0, ('B', 'R'): 2.0},
 {('B',): 3.0, ('B', 'R'): 6.0, ('B', 'R', 'R'): 6.0},
 {('B',): 4.0,
  ('B', 'R'): 12.0,
  ('B', 'R', 'R'): 24.0,
  ('B', 'R', 'R', 'R'): 24.0},
 {('B',): 5.0,
  ('B', 'R'): 20.0,
  ('B', 'R', 'R'): 60.0,
  ('B', 'R', 'R', 'R'): 120.0,
  ('B', 'R', 'R', 'R', 'R'): 120.0},
 {('B',): 6.0,
  ('B', 'R'): 30.0,
  ('B', 'R', 'R'): 120.0,
  ('B', 'R', 'R', 'R'): 360.0,
  ('B', 'R', 'R', 'R', 'R'): 720.0,
  ('B', 'R', 'R', 'R', 'R', 'R'): 720.0},
 {('B',): 7.0,
  ('B', 'R'): 42.0,
  ('B', 'R', 'R'): 210.0,
  ('B', 'R', 'R', 'R'): 840.0,
  ('B', 'R', 'R', 'R', 'R'): 2520.0,
  ('B', 'R', 'R', 'R', 'R', 'R'): 5040.0,
  ('B', 'R', 'R', 'R', 'R', 'R', 'R'): 5040.0}]

In [6]:
# use my util function to achieve the same

In [7]:
sources_using_util_fn = get_source_analysis_for_one_model_spec(
    num_recurrent_layer=1, num_cls=7, readout_type=None, return_raw=True
)

In [8]:
# same
assert [s.source_list for s in sources_using_util_fn] == [s.source_list for s in sources]

In [9]:
(lambda: 1.0)()

1.0

In [10]:
# let's see different readout types

def check_different_readout_type():
    scale_map = {
        # a default dict won't work!!! because .get() in defaultdict is the same is regular dict.
        f's{t}': 1.0 for t in range(1, 7+1)
    }
    
    for readout_type in ('inst-last', 'cm-last', 'inst-avg', 'cm-avg'):
        src_this = get_source_analysis_for_one_model_spec(
            num_recurrent_layer=1, num_cls=7, readout_type=readout_type
        )
        print(readout_type)
        print(src_this.unique_convs)
        print(src_this.source_list)
        print(src_this.evaluate(scale_map))
        print(src_this.evaluate(scale_map, {'B': 1.0, 'R': 2.0}))

In [11]:
check_different_readout_type()
# looks good!

inst-last
{('B', 'R', 'R', 'R', 'R', 'R', 'R'), ('B', 'R', 'R', 'R', 'R', 'R'), ('B', 'R', 'R', 'R', 'R'), ('B', 'R'), ('B', 'R', 'R'), ('B',), ('B', 'R', 'R', 'R')}
[{'conv': ('B',), 'scale': ('s7',)}, {'conv': ('B', 'R'), 'scale': ('s6', 's7')}, {'conv': ('B', 'R', 'R'), 'scale': ('s5', 's6', 's7')}, {'conv': ('B', 'R', 'R', 'R'), 'scale': ('s4', 's5', 's6', 's7')}, {'conv': ('B', 'R', 'R', 'R', 'R'), 'scale': ('s3', 's4', 's5', 's6', 's7')}, {'conv': ('B', 'R', 'R', 'R', 'R', 'R'), 'scale': ('s2', 's3', 's4', 's5', 's6', 's7')}, {'conv': ('B', 'R', 'R', 'R', 'R', 'R', 'R'), 'scale': ('s1', 's2', 's3', 's4', 's5', 's6', 's7')}]
{('B',): 1.0, ('B', 'R'): 1.0, ('B', 'R', 'R'): 1.0, ('B', 'R', 'R', 'R'): 1.0, ('B', 'R', 'R', 'R', 'R'): 1.0, ('B', 'R', 'R', 'R', 'R', 'R'): 1.0, ('B', 'R', 'R', 'R', 'R', 'R', 'R'): 1.0}
{('B',): 1.0, ('B', 'R'): 2.0, ('B', 'R', 'R'): 4.0, ('B', 'R', 'R', 'R'): 8.0, ('B', 'R', 'R', 'R', 'R'): 16.0, ('B', 'R', 'R', 'R', 'R', 'R'): 32.0, ('B', 'R', 'R', 'R',